# **PROYECTO SCHUTZ EN PYTHON PARA CREACIÓN DE DASHBOARD 2 - PARTE 1**

## 1.- IMPORTACIÓN DE LIBRERÍAS PARA EL PROYECTO

In [100]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from io import StringIO
from itertools import zip_longest
import io
import requests
from io import BytesIO
import pandas as pd
import chardet
import re
from oauth2client.service_account import ServiceAccountCredentials
import gspread
from gspread_dataframe import set_with_dataframe
from urllib.request import FancyURLopener
from urllib.request import urlopen
import urllib.request
import xml.etree.ElementTree as ET
import numpy as np
import datetime
import ssl
import warnings
from datetime import datetime, timedelta

In [101]:
pd.options.display.max_rows = None
pd.options.display.float_format = '{:.2f}'.format

warnings.filterwarnings("ignore", category=FutureWarning, message="Inferring datetime64.*")

## 2.- LECTURA DE DATAFRAMES

### 2.1 LECTURA "EMPLEADOS"

In [102]:
url = 'http://asp.maximise.cl/query.ashx?alias=gsea&user=excel_bd&password=excel_bd&filename=Gestion%5C%5CListo%20de%20Empleados%20Clasificados.txt'

# Descargar el archivo XML desde la URL
with urllib.request.urlopen(url) as response:
    xml_data = response.read()

# Parsear el archivo XML
root = ET.fromstring(xml_data)

# Crear una lista de diccionarios con los datos
data = []
for child in root:
    row = {}
    for subchild in child:
        row[subchild.tag] = subchild.text
    data.append(row)

# Convertir la lista de diccionarios en un dataframe de Pandas
df_empleados = pd.DataFrame(data)

# Imprimir el dataframe
df_empleados.to_csv('archivos_respaldo_dash2/df_empleados.csv', index = False)
df_empleados.sample(5)

,IDEMPLEADO,RUT,Nombre,apellidopaterno,apellidomaterno,idtipoempleado
758,20238934,20238934-1,Fernando Ignacio,Aguerrea,Inostroza,EX
582,18326071,18326071-5,Pedro Felipe,Valdebenito,Lopez,GE
780,20809495,20809495-5,Fernanda Jesús,Urria,Gonzalez,AF
769,20499219,20499219-3,Johann Osvaldo,Troncoso,Flores,TT
202,13705194,13705194-k,Hector E.,Serrano,Farias,GE


### 2.2 LECTURA "MAESTRO DE PRODUCTOS"

In [103]:
import urllib.request

url = 'https://asp.maximise.cl/query.ashx?alias=gsea&user=excel_bd&password=excel_bd&filename=Gestion\Maestro%20de%20Productos.txt'

# Disable SSL certificate verification (not recommended for production)
with urllib.request.urlopen(url, context=ssl._create_unverified_context()) as response:
    xml_data = response.read()

# Parsear el archivo XML
root = ET.fromstring(xml_data)

# Crear una lista de diccionarios con los datos
data = []
for child in root:
    row = {}
    for subchild in child:
        row[subchild.tag] = subchild.text
    data.append(row)

# Convertir la lista de diccionarios en un dataframe de Pandas
df_productos = pd.DataFrame(data)

# Imprimir el dataframe
df_productos.to_csv('archivos_respaldo_dash2/df_productos.csv', index = False)
df_productos.sample(5)

,Part_Number,Descripcion,Categoria,Familia,Costo_Promedio,Ultimo_x0020_Costo,Fecha_Ultima_Compra
1772,P08192024,Herramienta para bloqueo Avenger 13341-AK,P08,19,63015,86.7765934549486,2019-10-23T00:00:00-03:00
448,C08713020,DERIVADOR 5-1218 MHZ RIT ALAMBRICA 8D 12DB,C08,71,6396,6399,2023-04-06T00:00:00-03:00
1329,P07121017,Caja trasera para Anunciador Serial Hochiki FN...,P07,12,13872,25.3,2015-01-21T00:00:00-03:00
470,C08713712,DERIVADOR 5-2400 MHZ RIT INALAMBRICA 8D 12DB 5...,C08,71,6167,6167,2023-04-10T00:00:00-03:00
2076,P08202478,"Cañeria Pintada Ø8"" SCH10 A795 UL/FM",P08,20,146489,146489,2023-05-08T00:00:00-03:00


### 2.3 LECTURA "LIBRO MAYOR BASE"

In [104]:
archivo_csv = 'archivos_respaldo/df_libro_mayor5_base2.csv'  # Reemplaza 'ruta_del_archivo.csv' con la ubicación real de tu archivo CSV
# Leer el archivo CSV y crear un DataFrame
df_libro_mayor_base = pd.read_csv(archivo_csv)
print("DIMENSIONES DE df_libro_mayor_base:", df_libro_mayor_base.shape)
df_libro_mayor_base.sample(10)

DIMENSIONES DE df_libro_mayor_base: (340, 5)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE
97,P-1700,3. COSTO MANO DE OBRA,0,CD ORIZON,EBCO S.A.
88,P-1696,1. INGRESO,0,SANTA VICTORIA - TERRACORP,CONSTRUCTORA TERRACORP S.A.
150,P-1722,1. INGRESO,0,ECO CERRILLOS,CONSTRUCTORA ICF SPA
329,P-1726,4. COSTO SUPERVISIÓN,0,NaN,NaN
158,P-1725,2. COSTO MATERIALES,0,LO CAÑAS,CONSTRUCTORA BRISAS DE BATUCO
263,P-1675,3. COSTO MANO DE OBRA,0,PALENA II,CONSTRUCTORA MYA SPA.
226,P-1656,3. COSTO MANO DE OBRA,0,MIRADOR CLUB HÍPICO,CONSTRUCTORA VITAL EDIFICACION SPA
145,P-1720,3. COSTO MANO DE OBRA,0,NaN,NaN
107,P-1704,2. COSTO MATERIALES,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.
187,P-1620,1. INGRESO,0,"OF APOQUINDO- HABILITACIONES, INTEGRACION Y AD...",CONSTRUCTORA INARCO S.A.


### 2.4 LECTURA "LIBRO MAYOR"

In [105]:
# Se lee archivo que viene del tratamiento de datos del notebook Proy_Shutz_Dash1_v4, el cual ya clasifia el tipo de cuenta.

archivo_csv = 'archivos_respaldo/df_libro_mayor2.csv'  # Reemplaza 'ruta_del_archivo.csv' con la ubicación real de tu archivo CSV
# Leer el archivo CSV y crear un DataFrame
df_libro_mayor2 = pd.read_csv(archivo_csv)
print("DIMENSIONES DE df_libro_mayor:", df_libro_mayor2.shape)

df_libro_mayor2.sample(5)

DIMENSIONES DE df_libro_mayor: (24899, 18)


,Unnamed: 0,Codigo_proyecto,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Debito,Credito,Num_Referencia,Debito_2,Credito_2,TOTAL,Tipo_cuenta
10902,2602.00,P-1692,4101010-000,2023,3,2023-03-02T00:00:00-03:00,CIN,48,Amarras plasticas 300mm,P-1692,I16431048,5100,0,49666,5100,0,-5100,COSTO MATERIALES
23741,7141.00,P-1718,4150002-000,2023,2,2023-02-28T00:00:00-03:00,AJC,1173,REMUNERACIONES DIRECTAS PROYECTO,P-1718,17163173,745232,0,NaN,745232,0,-745232,COSTO MANO DE OBRA
8853,553.00,P-1680,3101030-000,2023,5,2023-05-31T00:00:00-04:00,CVT,1580,EP N°14 - OBRA:EDIFICIO NUEVA UNO,P-1680,91300000-5,0,2852406,13856,0,2852406,2852406,INGRESO
7784,7784.00,P-OFICINA,4150002-000,2023,5,2023-05-31T00:00:00-04:00,AJC,1844,REMUNERACIONES DIRECTAS PROYECTO,P-Oficina,16670655,61587,0,NaN,61587,0,-61587,COSTO MANO DE OBRA
4040,4040.00,P-1680,4101010-000,2023,4,2023-04-13T00:00:00-03:00,CIN,453,Cable coaxial RG11,P-1680,T14382752,133920,0,50395,133920,0,-133920,COSTO MATERIALES


# AGREGAR LECTURAS!!!

## 3.- TRATAMIENTO DE DATAFRAMES

### 3.1- TRATAMIENTO 1 - EMPLEADOS

In [106]:
# Función para categorizar la columna 'idtipoempleado'

def categorizar_pl(idtipoempleado):
    if idtipoempleado =="GC" or idtipoempleado == "SC" or idtipoempleado == "TC":
        return 'CCDD'
    elif idtipoempleado =="GP" or idtipoempleado == "SP" or idtipoempleado == "TP":
        return 'PIPING'
    else:
        return 'OTRA'

# Crear la nueva columna 'idtipoempleado' aplicando la función
df_empleados['PRACTICA2'] = df_empleados['idtipoempleado'].apply(categorizar_pl)

# Convertir las columnas a mayúsculas
df_empleados.columns = df_empleados.columns.str.upper()

# Convertir todo el contenido a mayúsculas
df_empleados = df_empleados.apply(lambda x: x.str.upper() if x.dtype == "object" else x)


df_empleados.sample(5)

,IDEMPLEADO,RUT,NOMBRE,APELLIDOPATERNO,APELLIDOMATERNO,IDTIPOEMPLEADO,PRACTICA2
154,13056802,13056802-5,CARLOS ARTURO ANDRES,TORO,LEYTON,CO,OTRA
367,16424146,16424146-7,BRAULIO SEBASTIAN,PARRA,ROJAS,CD,OTRA
397,16743622,16743622-6,GASTON FIDEL,LAGOS,MENDEZ,CO,OTRA
643,18865948,18865948-9,ARIEL MARCELO,VILLANUEVA,ACUÑA,TT,OTRA
601,18496267,18496267-5,CHRISTIAN BYRON,MARTÍNEZ,VERGARA,GE,OTRA


In [111]:
# Visualización del dataframe formato PIVOT:
df_empleados_piv = pd.pivot_table(df_empleados, values=['IDEMPLEADO'], index=['PRACTICA2'], aggfunc= 'count', fill_value=0)
df_empleados_piv

,IDEMPLEADO
PRACTICA2,
OTRA,896
PIPING,3


### 3.2- TRATAMIENTO 2 - MAESTRO DE PRODUCTOS

In [112]:
# Función para categorizar la columna 'PRACTICA'

def categorizar_pl(referencia):
    if referencia.startswith("C02") or referencia.startswith("C03") or referencia.startswith("C05") or referencia.startswith("C06") or referencia.startswith("C08"):
        return 'TELECOMUNICACIONES'
    elif referencia.startswith("C04") or referencia.startswith("C07") or referencia.startswith("C05") or referencia.startswith("I16") or referencia.startswith("O1950") or referencia.startswith("P07") or referencia.startswith("S09") or referencia.startswith("S10") or referencia.startswith("S10") or referencia.startswith("S11") or referencia.startswith("S12") or referencia.startswith("S13") or referencia.startswith("T14"):
        return 'CCDD'
    elif referencia.startswith("I15") or referencia.startswith("O1951") or referencia.startswith("C05") or referencia.startswith("C06") or referencia.startswith("P08"):
        return 'PIPING'
    else:
        return 'OTRA'

# Crear la nueva columna 'idtipoempleado' aplicando la función
df_productos['PRACTICA'] = df_productos['Part_Number'].apply(categorizar_pl)

# Convertir las columnas a mayúsculas
df_productos.columns = df_productos.columns.str.upper()

# Convertir todo el contenido a mayúsculas
df_productos = df_productos.apply(lambda x: x.str.upper() if x.dtype == "object" else x)

df_productos.sample(10)

df_productos.sample(5)

,PART_NUMBER,DESCRIPCION,CATEGORIA,FAMILIA,COSTO_PROMEDIO,ULTIMO_X0020_COSTO,FECHA_ULTIMA_COMPRA,PRACTICA
2632,S11301686,RECEPTOR INALAMBRICO DSC PC5132-433 COMPATIBLE...,S11,30,20925,20925,2019-02-15T00:00:00-03:00,CCDD
1371,P07121776,DETECTOR DE TEMPERATURA NOTIFIER 5601P CONVENC...,P07,14,6886,10.3379820755708,2018-08-28T00:00:00-03:00,CCDD
2239,P08212462,KIT BOMBA ELECTRICA 500-130 + JOCKEY + MEDIDOR...,P08,21,12621300,19450,2021-04-26T00:00:00-03:00,PIPING
2001,P08201586,"CAÑERIA NEGRA Ø10"" SCH10 A795",P08,20,188792,188792,2023-04-13T00:00:00-03:00,PIPING
707,H21581420,"SERRUCHO 20""",H21,58,2681,2680.67,2019-08-28T00:00:00-03:00,OTRA


### 3.3- TRATAMIENTO 3 - LIBRO MAYOR BASE

In [113]:
df_libro_mayor_base.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE
250,P-1668,4. COSTO SUPERVISIÓN,0,QUILLAYES PLAZA,E.MOLINA MOREL CONSTRUCTORA S.A.
94,P-1699,3. COSTO MANO DE OBRA,0,VILLA PANAMERICANA LOTE B,EMPRESA CONSTRUCTORA DLP S.A.
337,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL
220,P-1649,3. COSTO MANO DE OBRA,0,INACAP VITACURA E2,CONSTRUCTORA CUEVAS Y PURCELL S.A.
164,P-1727,2. COSTO MATERIALES,0,NaN,NaN


### 3.4- TRATAMIENTO 2 - LIBRO MAYOR

In [114]:
# Convertir las columnas a mayúsculas
df_libro_mayor2.columns = df_libro_mayor2.columns.str.upper()

# Convertir todo el contenido a mayúsculas
df_libro_mayor2 = df_libro_mayor2.apply(lambda x: x.str.upper() if x.dtype == "object" else x)

df_libro_mayor2.sample(5)

,UNNAMED: 0,CODIGO_PROYECTO,CTA_CONTABLE,AÑO,MES,FECHA,TIPO_COMP,NUM_COMP,GLOSA,PRIMER_ANALISIS,REFERENCIA,DEBITO,CREDITO,NUM_REFERENCIA,DEBITO_2,CREDITO_2,TOTAL,TIPO_CUENTA
22012,5412.00,P-1695,4101010-000,2023,5,2023-05-11T00:00:00-03:00,CIN,568,AMARRAS PLASTICAS 300MM,P-1695,I16431048,5100,0,51098,5100,0,-5100,COSTO MATERIALES
8212,8212.00,P-OFICINA,4150002-003,2023,4,2023-04-30T00:00:00-03:00,AJC,1681,REMUNERACIONES DIRECTAS PROYECTO POR DISTRIBUIR,P-OFICINA,19919528,48706,0,NaN,48706,0,-48706,OTRA
19315,2715.00,P-1688,4101010-000,2023,3,2023-03-07T00:00:00-03:00,CIN,172,"DISCO CORTE 4 1/2"" METAL",P-1688,I17440838,3040,0,49739,3040,0,-3040,COSTO MATERIALES
10442,2142.00,P-1692,4101010-000,2023,2,2023-02-13T00:00:00-03:00,CIN,323,DERIVADOR TAP 4 SAL 23DB 5-1200MHZ ALAMBRICA,P-1692,C08712744,50928,0,49415,50928,0,-50928,COSTO MATERIALES
137,137.00,P-1695,3101030-000,2023,2,2023-02-16T00:00:00-03:00,CVT,379,EP 299-49-6 OBRA 299 EDIFICIO HOY ESTORIL,P-1695,84102200-9,0,2256874,13430,0,2256874,2256874,INGRESO


#### 3.4.1 - MERGE LIBRO MAYOR <---- MAESTRO DE PRODUCTOS 

#### Se realiza merge con maestro de productos para distinguir el tipo de costo de materiales

In [115]:
print("-----------------------------------------------------------------")
print("3.4.1 - MERGE LIBRO MAYOR <---- MAESTRO DE PRODUCTOS")
print("-----------------------------------------------------------------")
print("DIMENSIONES ANTES DE MERGE DATAFRAME   : df_libro_mayor2   : ", df_libro_mayor2.shape)

df_libro_mayor2_1 = pd.merge(df_libro_mayor2, df_productos, left_on='REFERENCIA', right_on='PART_NUMBER', how='left')

print("DIMENSIONES DESPUÉS DE MERGE DATAFRAME : df_libro_mayor2_1 : ", df_libro_mayor2_1.shape)
df_libro_mayor2_1.sample(5)

-----------------------------------------------------------------
3.4.1 - MERGE LIBRO MAYOR <---- MAESTRO DE PRODUCTOS
-----------------------------------------------------------------
DIMENSIONES ANTES DE MERGE DATAFRAME   : df_libro_mayor2   :  (24899, 18)
DIMENSIONES DESPUÉS DE MERGE DATAFRAME : df_libro_mayor2_1 :  (24899, 26)


,UNNAMED: 0,CODIGO_PROYECTO,CTA_CONTABLE,AÑO,MES,FECHA,TIPO_COMP,NUM_COMP,GLOSA,PRIMER_ANALISIS,...,TOTAL,TIPO_CUENTA,PART_NUMBER,DESCRIPCION,CATEGORIA,FAMILIA,COSTO_PROMEDIO,ULTIMO_X0020_COSTO,FECHA_ULTIMA_COMPRA,PRACTICA
11067,2767.00,P-1686,4101010-000,2023,3,2023-03-07T00:00:00-03:00,CIN,188,CONTROL REMOTO GARRISON 1 CANAL,P-1686,...,-17992,COSTO MATERIALES,S12341189,CONTROL REMOTO GARRISON 1 CANAL,S12,34,7688,7226,2023-06-29T00:00:00-04:00,CCDD
20798,4198.00,P-1726,4101010-000,2023,4,2023-04-17T00:00:00-03:00,CIN,604,"DISCO CORTE 4 1/2"" METAL",P-1726,...,-18950,COSTO MATERIALES,I17440838,"DISCO CORTE 4 1/2"" METAL",I17,44,382,378.15,2023-06-29T00:00:00-04:00,OTRA
8394,94.00,P-1721,3101030-000,2023,1,2023-01-31T00:00:00-03:00,AJC,1686,VENTAS DE PROYECTO,P-1721,...,-7184888,INGRESO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5164,5164.00,P-1715,4101010-000,2023,5,2023-05-08T00:00:00-03:00,CIN,188,"UNION COPLA 8"" RANURADA FLEXIBLE",P-1715,...,-48685,COSTO MATERIALES,P08200564,"UNION COPLA 8"" RANURADA FLEXIBLE",P08,20,9737,9737,2023-05-08T00:00:00-03:00,PIPING
9028,728.00,P-1680,3401002-000,2023,5,2023-05-31T00:00:00-04:00,AJC,1631,AJUSTE POR ANTICIPOS Y RETENCIONES,P-1680,...,950802,INGRESO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 3.4.2 - MERGE LIBRO MAYOR <---- EMPLEADOS

In [126]:
print("-----------------------------------------------------------------")
print("3.4.2 - MERGE LIBRO MAYOR <---- EMPLEADOS")
print("-----------------------------------------------------------------")
print("DIMENSIONES ANTES DE MERGE DATAFRAME   : df_libro_mayor2_1  : ", df_libro_mayor2_1.shape)

df_libro_mayor2_2 = pd.merge(df_libro_mayor2_1, df_empleados, left_on='REFERENCIA', right_on='IDEMPLEADO', how='left')
# Se seleccionan columnas utiles:
df_libro_mayor2_2 = df_libro_mayor2_2[["CODIGO_PROYECTO","CTA_CONTABLE","AÑO","MES","TIPO_COMP","GLOSA","REFERENCIA","DEBITO_2","CREDITO_2","TOTAL","TIPO_CUENTA","PART_NUMBER","CATEGORIA","FAMILIA","PRACTICA","IDEMPLEADO","NOMBRE","APELLIDOPATERNO","IDTIPOEMPLEADO","PRACTICA2"]]

print("DIMENSIONES DESPUÉS DE MERGE DATAFRAME : df_libro_mayor2_2  : ", df_libro_mayor2_2.shape)
df_libro_mayor2_2.sample(30)

-----------------------------------------------------------------
3.4.2 - MERGE LIBRO MAYOR <---- EMPLEADOS
-----------------------------------------------------------------
DIMENSIONES ANTES DE MERGE DATAFRAME   : df_libro_mayor2_1  :  (24899, 26)
DIMENSIONES DESPUÉS DE MERGE DATAFRAME : df_libro_mayor2_2  :  (24899, 20)


,CODIGO_PROYECTO,CTA_CONTABLE,AÑO,MES,TIPO_COMP,GLOSA,REFERENCIA,DEBITO_2,CREDITO_2,TOTAL,TIPO_CUENTA,PART_NUMBER,CATEGORIA,FAMILIA,PRACTICA,IDEMPLEADO,NOMBRE,APELLIDOPATERNO,IDTIPOEMPLEADO,PRACTICA2
23593,P-1680,4150002-000,2023,1,AJC,REMUNERACIONES DIRECTAS PROYECTO,18277810,551630,0,-551630,COSTO MANO DE OBRA,NaN,NaN,NaN,NaN,18277810,ALEJANDRO JAVIER,CANDIA,SX,OTRA
17374,P-1716,4101010-000,2023,1,CIN,"SOPORTE PERA 2"" UL/FM",P08200266,5808,0,-5808,COSTO MATERIALES,P08200266,P08,20,PIPING,NaN,NaN,NaN,NaN,NaN
22074,P-1699,4101010-000,2023,5,CIN,CABLE PE 0.9 X 2,T14380056,87200,0,-87200,COSTO MATERIALES,T14380056,T14,38,CCDD,NaN,NaN,NaN,NaN,NaN
15880,P-1716,4150002-000,2023,4,AJC,REMUNERACIONES DIRECTAS PROYECTO,26537138,61302,0,-61302,COSTO MANO DE OBRA,NaN,NaN,NaN,NaN,26537138,LUIS EDUARDO,MIJARES,SX,OTRA
10652,P-1712,4101010-000,2023,2,CIN,COPLA EF PE100 PN16 200MM,O19501068,53626,0,-53626,COSTO MATERIALES,O19501068,O19,50,CCDD,NaN,NaN,NaN,NaN,NaN
12618,P-1697,4101010-000,2023,4,CIN,BANDEJA SIMPLE PARA RACK,C02030694,19174,0,-19174,COSTO MATERIALES,C02030694,C02,03,TELECOMUNICACIONES,NaN,NaN,NaN,NaN,NaN
21852,P-1704,4101010-000,2023,5,CIN,"CAÑERIA PINTADA Ø2"" SCH10 A795 UL/FM",P08200082,2023000,0,-2023000,COSTO MATERIALES,P08200082,P08,20,PIPING,NaN,NaN,NaN,NaN,NaN
18264,P-1718,4101010-000,2023,1,CIN,"SOPORTE PERA 3"" UL/FM",P08202222,69918,0,-69918,COSTO MATERIALES,P08202222,P08,20,PIPING,NaN,NaN,NaN,NaN,NaN
9643,P-1704,4101010-000,2023,1,CIN,"SOPORTE ANTISISMICO ANCLAJE 1/2"" UL FM T",P08201874,354840,0,-354840,COSTO MATERIALES,P08201874,P08,20,PIPING,NaN,NaN,NaN,NaN,NaN
5613,P-1712,4101010-000,2023,5,CIN,"SOPORTE PERA 1"" UL/FM",P08200265,6300,0,-6300,COSTO MATERIALES,P08200265,P08,20,PIPING,NaN,NaN,NaN,NaN,NaN


In [127]:
# Imprimir el dataframe
df_libro_mayor2_2.to_csv('archivos_respaldo_dash2/df_libro_mayor2_2.csv', index = False)

## AQUI VOY
